In [8]:
!pip install elasticsearch
!pip install --upgrade elasticsearch

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [9]:
from elasticsearch import Elasticsearch

In [11]:
from elasticsearch import Elasticsearch
# Define your cloud credentials
ELASTIC_CLOUD_ID = "API"
ELASTIC_API_KEY = "API"

# Initialize Elasticsearch client
es_client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)
# Get all indices
indices = es_client.cat.indices(format="json")
print("All indices:", indices)


All indices: [{'health': 'green', 'status': 'open', 'index': '.internal.alerts-ml.anomaly-detection.alerts-default-000001', 'uuid': 'mtpTYkQHTEG_Duzcxik0vg', 'pri': '1', 'rep': '1', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '498b', 'pri.store.size': '249b', 'dataset.size': '249b'}, {'health': 'green', 'status': 'open', 'index': '.ds-logs-enterprise_search.audit-default-2025.01.29-000001', 'uuid': 'BTceimaMRTOTipptBxL-dw', 'pri': '1', 'rep': '1', 'docs.count': '8', 'docs.deleted': '0', 'store.size': '138.4kb', 'pri.store.size': '76.7kb', 'dataset.size': '76.7kb'}, {'health': 'green', 'status': 'open', 'index': '.internal.alerts-observability.threshold.alerts-default-000001', 'uuid': 'nTqwnFXyQLqzJiW82gJfVg', 'pri': '1', 'rep': '1', 'docs.count': '0', 'docs.deleted': '0', 'store.size': '498b', 'pri.store.size': '249b', 'dataset.size': '249b'}, {'health': 'green', 'status': 'open', 'index': '.internal.alerts-observability.metrics.alerts-default-000001', 'uuid': '83hvSu1hR8-Yr3

In [12]:
print(es_client.info())

{'name': 'instance-0000000000', 'cluster_name': 'f5b2949cda5442aea62038aae80cc2e7', 'cluster_uuid': 'bmrFfMEBTpCfeJreYzhQBQ', 'version': {'number': '8.17.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd4b391d925c31d262eb767b8b2db8f398103f909', 'build_date': '2025-01-10T10:08:26.972230187Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [13]:
!pip install elasticsearch openai

Defaulting to user installation because normal site-packages is not writeable


In [1]:
from elasticsearch import Elasticsearch
from openai import OpenAI
import json

# Define your cloud credentials
ELASTIC_CLOUD_ID = "API"
ELASTIC_API_KEY = "API"
OPENAI_API_KEY = "API"

# Initialize Elasticsearch client
es_client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)
# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Function to index documents in Elasticsearch
def index_document(index_name, doc_id, document):
    es_client.index(index=index_name, id=doc_id, document=document)
    print(f"Indexed document {doc_id}")

# Function to retrieve relevant documents from Elasticsearch
def retrieve_documents(index_name, query_text, top_k=3):
    response = es_client.search(
        index=index_name,
        body={
            "query": {
                "match": {
                    "content": query_text
                }
            },
            "size": top_k
        }
    )
    return [hit["_source"]["content"] for hit in response["hits"]["hits"]]

# Function to generate an answer using OpenAI
def generate_answer(query, retrieved_docs):
    context = "\n".join(retrieved_docs)
    messages = [
        {"role": "system", "content": "You are a helpful AI assistant answering based on retrieved documents."},
        {"role": "user", "content": f"Use the following documents to answer the query:\n\n{context}\n\nQuery: {query}"}
    ]

    response = openai_client.chat.completions.create(
        model="gpt-4",  # Ensure you're using a chat model
        messages=messages,
        max_tokens=200
    )

    return response.choices[0].message.content.strip()

# Example Usage
if __name__ == "__main__":
    index_name = "rag_dataset"

    # Sample documents (indexing)
    documents = [
        {"id":'01',"content":"""The quick brown fox jumps over the lazy dog.
        A fox is a small carnivorous mammal known for its cunning behavior.
        Foxes are part of the Canidae family, closely related to dogs.
        Mammals are warm-blooded animals that have fur or hair and produce milk to feed their young.
        They are highly diverse and can be found in various environments across the globe,Elephants: Large terrestrial animals known for their intelligence, 
        long trunks, and strong familial bonds,Tigers: Apex predators with striped fur, 
        native to Asia and known for their solitary behavior,Dolphins: Highly intelligent marine mammals that communicate using echolocation,
        Birds are warm-blooded creatures characterized by feathers, beaks, and the ability to lay eggs. 
        Many birds can fly, although some, like ostriches and penguins, are flightless,Eagles: Birds of prey with sharp talons and keen eyesight,Parrots: Colorful birds known for their ability to mimic human speech,
        Penguins: Flightless birds adapted to live in cold climates, especially in Antarctica,Reptiles are cold-blooded animals with scales or scutes. Most reptiles lay eggs, although some, 
        like certain snakes, give birth to live young,Crocodiles: Large, aquatic reptiles found in rivers and wetlands.Lizards: Diverse reptiles that range from small geckos to large Komodo dragons.Turtles: Reptiles with protective shells, 
        some of which live in aquatic environments.Amphibians are cold-blooded animals that typically live both in water and on land during their life cycle,Frogs: Amphibians known for their jumping abilities and croaking sounds.
        Salamanders: Lizard-like amphibians with smooth skin and the ability to regenerate limbs.Toads: Amphibians similar to frogs but with drier, 
        bumpier skin.Fish are cold-blooded aquatic animals that breathe through gills and have fins for swimming. They can be found in both freshwater and saltwater environments.
        Sharks: Predatory fish with sharp teeth and a cartilaginous skeleton.Clownfish: Brightly colored fish known for their symbiotic relationship with sea anemones.
        Salmon: Fish that migrate upstream to spawn in freshwater rivers.Insects are invertebrates with a three-part body structure (head, thorax, abdomen), six legs, and usually wings. 
        Bees: Pollinators essential for many ecosystems and agricultural crops.Butterflies: Insects known for their colorful wings and metamorphic life cycle.Ants: Social insects that live in colonies and exhibit complex behaviors.
        Cheetahs are the fastest land animals, capable of reaching speeds up to 70 mph.Octopuses have three hearts and blue blood.Kangaroos can’t walk backward due to the structure of their legs and tail.Hummingbirds are the only birds that can fly backward.
        Snakes can sense heat through specialized pits near their nostrils, allowing them to detect prey.Animals are a diverse group of living organisms that inhabit nearly every corner of the planet, from the deepest oceans to the highest mountains.
        They range in size from microscopic creatures, like plankton, to massive mammals, such as whales and elephants. Animals play a crucial role in maintaining ecological balance, serving as pollinators, predators, prey, and more in their respective ecosystems.
        They exhibit incredible adaptations to survive in their environments, such as the camouflage of chameleons, the migration of birds, and the echolocation of bats.
        With over a million species identified, animals continue to fascinate and inspire humans, reminding us of the beauty and complexity of life on Earth.
        The animal kingdom is incredibly diverse, encompassing a wide range of creatures adapted to various habitats and diets. Aquatic animals, such as fish, whales, and octopuses, thrive in water environments, relying on specialized adaptations like gills or streamlined bodies for survival.
        Reptiles, including snakes, turtles, and crocodiles, are cold-blooded creatures often found in both aquatic and terrestrial habitats, known for their scaly skin and ability to lay eggs.
        Mammals, such as lions, dolphins, and humans, are warm-blooded and distinguished by their ability to produce milk for their young.
        Among mammals and other creatures, dietary preferences vary widely: carnivores, like tigers and sharks, consume meat as their primary food source; herbivores, such as deer, elephants, and giraffes, feed on plants; and omnivores, like bears, raccoons, and pigs, have versatile diets that include both plant and animal matter.
        Together, these animals play essential roles in maintaining the balance of ecosystems across the planet.The Amazon Rainforest, often referred to as the lungs of the Earth,is the largest tropical rainforest in the world, spanning over 5.5 million square kilometers across South America, primarily in Brazil, Peru, and Colombia.
        It is a vital ecological treasure, home to an extraordinary diversity of flora and fauna, including jaguars, macaws, and thousands of tree species. 
        The Amazon plays a crucial role in regulating the Earth's climate by absorbing vast amounts of carbon dioxide and producing oxygen. Additionally, it supports millions of indigenous people who rely on its resources for sustenance and cultural practices. However,
        the Amazon faces significant threats from deforestation, illegal logging, and climate change, which endanger its unique biodiversity and global environmental stability.
        Conservation efforts are critical to protecting this irreplaceable natural wonder for future generations.koalas, are iconic marsupials native to Australia, known for their cuddly appearance and unique lifestyle.
        These tree-dwelling herbivores primarily feed on eucalyptus leaves, which provide both their food and water needs. Despite their laid-back demeanor and long hours of sleep—up to 20 hours a day—koalas are incredibly adapted for life in the trees, with sharp claws and strong limbs that help them climb and grip branches.
        They are solitary animals with a specialized digestive system that allows them to process the toxic compounds in eucalyptus leaves. Unfortunately, koalas face threats from habitat destruction, climate change, and diseases like chlamydia, making them a species in need of conservation efforts.
        Beloved worldwide, koalas symbolize the rich biodiversity of Australia's unique ecosystems.Fossils of dinosaurs provide an extraordinary window into Earth's prehistoric past, showcasing the incredible diversity of these ancient creatures that dominated the planet for over 160 million years. 
        Found embedded in rock layers across the globe, dinosaur fossils include bones, teeth, footprints, and even skin impressions or traces of feathers. These fossils reveal critical insights into their size, behavior, diets, and evolutionary adaptations.
        Paleontologists use fossilized remains to reconstruct entire skeletons and understand how dinosaurs interacted with their environment. Fossil discoveries, such as those of Tyrannosaurus rex, Triceratops, and Velociraptor, continue to captivate scientists and the public alike, 
        highlighting how life on Earth has evolved dramatically over millions of years.The Ice Age refers to a period in Earth's history marked by prolonged global cooling, during which vast ice sheets and glaciers covered significant portions of the planet.
        These periods, also known as glacial periods, alternated with warmer interglacial phases in cycles spanning tens of thousands of years. The most recent Ice Age, known as the Quaternary Glaciation, began approximately 2.58 million years ago and is ongoing, 
        with the last major glacial period peaking around 20,000 years ago before the ice began retreating. During the Ice Age, massive glaciers shaped the Earth's surface, carving valleys, creating lakes, and altering ecosystems. 
        This era also influenced the evolution and migration of many species, including early humans, who adapted to the harsh climates by developing tools, clothing, and strategies for survival. The remnants of this icy epoch, such as the polar ice caps and alpine glaciers, continue to shape Earth's environment today.
        The early people, often referred to as prehistoric humans, were the first to emerge on Earth, adapting to their environment through innovation and survival skills. They lived in small groups, relying on hunting, gathering, and basic tools to meet their needs. 
        These early humans were primarily nomadic, moving from place to place in search of food and shelter. Over time, they developed language, social structures, and early forms of art, such as cave paintings, that reflected their experiences and beliefs. 
        Their ability to adapt to changing climates, discover fire, and invent rudimentary tools marked the beginnings of human civilization, setting the stage for the development of agriculture, permanent settlements, and the rise of more complex societies."""},
    ]
    
    # Index the documents
    for doc in documents:
        index_document(index_name, doc["id"], doc)

    # Query for retrieval
    query_text = "Is Fish are cold-blooded animal or reptile ?"
    retrieved_docs = retrieve_documents(index_name, query_text)

    # Generate augmented answer
    answer = generate_answer(query_text, retrieved_docs)
    print("\n--- AI-Generated Answer ---")
    print(answer)

Indexed document 01

--- AI-Generated Answer ---
Fish are cold-blooded aquatic animals, as mentioned in the document. They are not reptiles. Reptiles are another group of cold-blooded animals known for their scaly skin and ability to lay eggs, and they are typically found in both aquatic and terrestrial habitats.


In [16]:
!pip install elasticsearch openai pypdf

Defaulting to user installation because normal site-packages is not writeable


In [18]:
from elasticsearch import Elasticsearch
from openai import OpenAI
import json
import os
import pypdf  # Library for extracting text from PDFs
import numpy as np

# Define your cloud credentials
ELASTIC_CLOUD_ID = "api"
ELASTIC_API_KEY = "api"
OPENAI_API_KEY = "Api"

# Initialize Elasticsearch client
es_client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as f:
        reader = pypdf.PdfReader(f)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

# Function to generate embeddings using OpenAI
def generate_embedding(text):
    response = openai_client.embeddings.create(
        model="text-embedding-ada-002",  # Use OpenAI's embedding model
        input=text
    )
    return response.data[0].embedding  # Returns a 1536-dimensional embedding

# Function to index PDF content with vector embeddings in Elasticsearch
def index_pdf(index_name, doc_id, pdf_path):
    text = extract_text_from_pdf(pdf_path)
    if text:
        embedding = generate_embedding(text)  # Generate embedding for text
        document = {
            "id": doc_id,
            "content": text,
            "embedding": embedding  # Store vector embedding
        }
        es_client.index(index=index_name, id=doc_id, document=document)
        print(f"Indexed PDF: {pdf_path} (ID: {doc_id})")
    else:
        print(f"Could not extract text from: {pdf_path}")

# Function to retrieve relevant documents using vector search
def retrieve_documents(index_name, query_text, top_k=3):
    # Generate embedding for query
    query_embedding = generate_embedding(query_text)
    
    # Perform hybrid search: full-text + vector similarity
    response = es_client.search(
        index=index_name,
        knn={
            "field": "embedding",
            "query_vector": query_embedding,
            "k": top_k,
            "num_candidates": 10  # Adjust for better accuracy
        },
        query={
            "match": {
                "content": query_text
            }
        },
        size=top_k
    )
    return [hit["_source"]["content"] for hit in response["hits"]["hits"]]

# Function to generate an answer using OpenAI
def generate_answer(query, retrieved_docs):
    context = "\n".join(retrieved_docs)
    messages = [
        {"role": "system", "content": "You are an AI assistant that answers queries using provided documents."},
        {"role": "user", "content": f"Use the following documents to answer the query:\n\n{context}\n\nQuery: {query}\nAnswer:"}
    ]
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200
    )
    return response.choices[0].message.content.strip()

# Example Usage
if __name__ == "__main__":
    index_name = "health_dataset"

    # Sample PDF file paths (change to actual paths)
    pdf_files = ["/home/kiwitech/Downloads/symptom_condition_treatment.pdf",
                 "/home/kiwitech/Downloads/Health_data - symptomps.pdf"]

    # Index PDF documents with vector embeddings
    for idx, pdf_path in enumerate(pdf_files):
        if os.path.exists(pdf_path):
            index_pdf(index_name, str(idx + 1), pdf_path)
        else:
            print(f"File not found: {pdf_path}")

    # Query for retrieval
    query_text = "what are the symptoms of Vertigo?"
    retrieved_docs = retrieve_documents(index_name, query_text)

    # Generate AI response
    answer = generate_answer(query_text, retrieved_docs)

    print("\n--- Retrieved Documents ---")
    print(json.dumps(retrieved_docs, indent=2))

    print("\n--- AI-Generated Answer ---")
    print(answer)

Indexed PDF: /home/kiwitech/Downloads/symptom_condition_treatment.pdf (ID: 1)
Indexed PDF: /home/kiwitech/Downloads/Health_data - symptomps.pdf (ID: 2)

--- Retrieved Documents ---
[
  "symptom_condition_treatment\nPage 1\nSymptom Treatment Condition\nFever Antiviral medications, rest, and hydration Influenza\nCough Bronchodilators and corticosteroids Asthma\nChest Pain Nitroglycerin, aspirin, and lifestyle changes Angina\nHeadache Pain relievers, triptans, and relaxation techniques Migraine\nSore Throat Throat lozenges and warm saline gargle Pharyngitis\nShortness of Breath Bronchodilators and oxygen therapy COPD\nNausea Antacids, PPIs, and bland diet Gastritis\nFatigue Iron supplements and dietary adjustments Anemia\nJoint Pain Anti-inflammatory medications and physical therapyArthritis\nDizziness Antihistamines, vestibular therapy Vertigo\nFever Antiviral medications, rest, and hydration Influenza\nCough Bronchodilators and corticosteroids Asthma\nChest Pain Nitroglycerin, aspirin, 

In [9]:
!pip install elasticsearch openai
!pip install elasticsearch openai pypdf
!!pip install tiktoken
!pip install --upgrade openai

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Attempting uninstall: openai
    Found existing installation: openai 1.59.7
    Uninstalling openai-1.59.7:
      Successfully uninstalled openai-1.59.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
camel-ai 0.2.11 requires docstring-parser<0.16,>=0.15, but you have docstring-parser 0.16 which is incompatible.
camel-ai 0.2.11 requires pydantic<2.10,>=1.9, but you have pydantic 2.10.5 whic

In [14]:
!pip install faiss-cpu rank-bm25 openai numpy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [1]:
import numpy as np
import faiss
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Sample corpus (you can use your own corpus)
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "A fast fox jumps over a sleeping dog.",
    "Artificial intelligence and machine learning are advancing rapidly.",
    "Python is a great programming language for AI and ML.",
    "Data science involves using statistical techniques to analyze data."
]
# Step 1: Sparse Search using TF-IDF
def sparse_search(query, documents, top_k=3):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    query_vec = vectorizer.transform([query])
    
    # Compute cosine similarity
    cos_sim = cosine_similarity(query_vec, tfidf_matrix)
    scores = cos_sim[0]
    
    # Get the top_k results
    top_indices = np.argsort(scores)[::-1][:top_k]
    return [(documents[i], scores[i]) for i in top_indices]

# Step 2: Vector Search using FAISS
def vector_search(query, documents, model, top_k=3):
    # Encode documents and query to vectors
    doc_embeddings = model.encode(documents)
    query_embedding = model.encode([query])
    
    # Convert embeddings to FAISS-compatible format (float32)
    doc_embeddings = np.array(doc_embeddings).astype(np.float32)
    query_embedding = np.array(query_embedding).astype(np.float32)
    
    # Build the FAISS index
    index = faiss.IndexFlatL2(doc_embeddings.shape[1])
    index.add(doc_embeddings)
    
    # Perform the search
    distances, indices = index.search(query_embedding, top_k)
    
    # Return the top_k results
    return [(documents[i], distances[0][idx]) for idx, i in enumerate(indices[0])]

# Step 3: Hybrid Search (Combining Sparse and Vector Search)
def hybrid_search(query, documents, model, top_k=3):
    # Get the sparse search results
    sparse_results = sparse_search(query, documents, top_k)
    
    # Get the vector search results
    vector_results = vector_search(query, documents, model, top_k)
    
    # Combine and rank by score (using a simple sum of both search results)
    combined_scores = {}
    
    # Adding sparse results
    for doc, score in sparse_results:
        combined_scores[doc] = combined_scores.get(doc, 0) + score
    
    # Adding vector results
    for doc, score in vector_results:
        combined_scores[doc] = combined_scores.get(doc, 0) + 1 / (score + 1e-9)  # Avoid division by zero
    
    # Sort results by combined score
    sorted_docs = sorted(combined_scores.items(), key=lambda x: x[1], reverse=True)
    return sorted_docs[:top_k]

# Example usage
query = "AI and machine learning advancements"

# Load SentenceTransformer model (this can be replaced with your own pre-trained model)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Perform hybrid search
results = hybrid_search(query, documents, model, top_k=3)

# Display results
print("Hybrid Search Results:")
for doc, score in results:
    print(f"Document: {doc}\nScore: {score}\n")

/home/kiwitech/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2025-02-03 13:07:07.137644: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 13:07:07.432691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-03 13:07:07.535063: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already bee

Hybrid Search Results:
Document: Artificial intelligence and machine learning are advancing rapidly.
Score: 2.439821247071796

Document: Python is a great programming language for AI and ML.
Score: 1.1714613370416944

Document: Data science involves using statistical techniques to analyze data.
Score: 0.7031776156010725



In [6]:
import numpy as np
import faiss
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from PyPDF2 import PdfReader

# Set API keys
ELASTIC_CLOUD_ID = "api"
ELASTIC_API_KEY = "api"
OPENAI_API_KEY = "API"
INDEX_NAME = "human_rights_document"

# Initialize Elasticsearch client
es_client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)
# Initialize SentenceTransformer model
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# Function to create an index in Elasticsearch
def create_index():
    """Creates an Elasticsearch index with text fields for BM25 search."""
    settings = {
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 0
        },
        "mappings": {
            "properties": {
                "text": {
                    "type": "text"
                }
            }
        }
    }
    es_client.indices.create(index=INDEX_NAME, body=settings, ignore=400)
    print(f"Index '{INDEX_NAME}' created (if not already existing).")

# Function to index documents into Elasticsearch
def index_document(doc_id, text):
    """Indexes a document in Elasticsearch."""
    document = {"text": text}
    es_client.index(index=INDEX_NAME, id=doc_id, body=document)
    print(f"Document {doc_id} indexed.")

# Function to perform sparse search using Elasticsearch (BM25)
def sparse_search(query, top_k=3):
    """Performs sparse search using Elasticsearch BM25 scoring."""
    query_body = {
        "size": top_k,
        "query": {
            "match": {
                "text": query
            }
        }
    }
    response = es_client.search(index=INDEX_NAME, body=query_body)
    results = [(hit["_source"]["text"], hit["_score"]) for hit in response["hits"]["hits"]]
    return results

# Function to perform vector search using FAISS and SentenceTransformers
def vector_search(query, documents, model, top_k=3):
    """Performs vector search using FAISS and SentenceTransformer model."""
    doc_embeddings = model.encode(documents)
    query_embedding = model.encode([query])
    
    # Convert embeddings to FAISS-compatible format (float32)
    doc_embeddings = np.array(doc_embeddings).astype(np.float32)
    query_embedding = np.array(query_embedding).astype(np.float32)
    
    # Build the FAISS index
    index = faiss.IndexFlatL2(doc_embeddings.shape[1])
    index.add(doc_embeddings)
    
    # Perform the search
    distances, indices = index.search(query_embedding, top_k)
    
    return [(documents[i], distances[0][idx]) for idx, i in enumerate(indices[0])]

# Hybrid search function combining sparse and vector search
def hybrid_search(query, documents, model, top_k=3):
   
    # Extract the most relevant snippet (specific answer) from each document
    results = []
    for doc, score in sorted_docs[:top_k]:
        snippet, snippet_score = extract_relevant_answer(query, doc)
        results.append((snippet, snippet_score))

    return results
     
# Function to extract the most relevant answer (snippet) for a given query
def extract_relevant_answer(query, document):
    """Extracts the most relevant sentence or passage from the document based on the query."""
    sentences = document.split(". ")  # Split the document into sentences
    
    # Encode the query and sentences using the sentence transformer model
    query_embedding = model.encode([query])[0]
    sentence_embeddings = model.encode(sentences)
    
    # Compute cosine similarity between the query and each sentence
    similarities = [np.dot(query_embedding, sentence_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(sentence_embedding))
                    for sentence_embedding in sentence_embeddings]
    
    # Find the sentence with the highest similarity
    most_relevant_idx = np.argmax(similarities)
    relevant_snippet = sentences[most_relevant_idx]
    snippet_score = similarities[most_relevant_idx]

    return relevant_snippet, snippet_score

# Function to extract text from a PDF document
def extract_text_from_pdf(pdf_file):
    """Extracts text from a PDF file using PyPDF2."""
    reader = PdfReader(pdf_file)
    text = ""
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

# Example usage
if __name__ == "__main__":
    create_index()

    # Extract text from PDFs
    pdf_files = ["/home/kiwitech/Downloads/human_rights.pdf"]
    documents = [extract_text_from_pdf(pdf) for pdf in pdf_files]

    # Index documents into Elasticsearch
    for idx, text in enumerate(documents):
        index_document(idx, text)

    # Hybrid search example
    query = "What is the role of the United Nations in human rights?"
    results = hybrid_search(query, documents, model, top_k=3)

    print("\nHybrid Search Results:")
    for snippet, score in results:
        print(f"Snippet: {snippet}\nScore: {score}\n")

/tmp/ipykernel_19253/3467556320.py:37: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es_client.indices.create(index=INDEX_NAME, body=settings, ignore=400)


Index 'human_rights_document' created (if not already existing).
Document 0 indexed.


NameError: name 'sorted_docs' is not defined

In [22]:
from elasticsearch import Elasticsearch
from openai import OpenAI
import json
import os
import pypdf  
import numpy as np

# Define your cloud credentials
ELASTIC_CLOUD_ID = "API"
ELASTIC_API_KEY = "API"
OPENAI_API_KEY = "API"

# Initialize Elasticsearch client
es_client = Elasticsearch(
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)
# Initialize OpenAI client
openai_client = OpenAI(api_key=OPENAI_API_KEY)

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as f:
        reader = pypdf.PdfReader(f)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])
    return text

# Function to generate embeddings using OpenAI
def generate_embedding(text):
    response = openai_client.embeddings.create(
        model="text-embedding-ada-002",  # Use OpenAI's embedding model
        input=text
    )
    return response.data[0].embedding  # Returns a 1536-dimensional embedding

# Function to index PDF content with vector embeddings in Elasticsearch
def index_pdf(index_name, doc_id, pdf_path):
    text = extract_text_from_pdf(pdf_path)
    if text:
        embedding = generate_embedding(text)  # Generate embedding for text
        document = {
            "id": doc_id,
            "content": text,
            "embedding": embedding  # Store vector embedding
        }
        es_client.index(index=index_name, id=doc_id, document=document)
        print(f"Indexed PDF: {pdf_path} (ID: {doc_id})")
    else:
        print(f"Could not extract text from: {pdf_path}")

# Function to retrieve relevant documents using hybrid search and return content with scores
def retrieve_documents(index_name, query_text, top_k=3):
    query_embedding = generate_embedding(query_text)

    response = es_client.search(
        index=index_name,
        knn={
            "field": "embedding",
            "query_vector": query_embedding,
            "k": top_k,
            "num_candidates": 10
        },
        query={
            "match": {
                "content": query_text
            }
        },
        size=top_k
    )
    retrieved_data = [
        {"content": hit["_source"]["content"], "score": hit["_score"]}
        for hit in response["hits"]["hits"]
    ]
    return retrieved_data  # Return documents with scores

# Function to generate an AI answer with retrieved snippets and scores
def generate_answer(query, retrieved_docs):
    # Prepare context from top-matching document snippets
    context = "\n\n".join(
        [f"Score: {doc['score']}\nSnippet: {doc['content'][:500]}" for doc in retrieved_docs]
    )
    messages = [
        {"role": "system", "content": "You are an AI assistant that answers queries using provided documents."},
        {"role": "user", "content": f"Use the following extracted snippets to answer the query:\n\n{context}\n\nQuery: {query}\nProvide a concise and accurate answer based only on the given snippets."}
    ]
    response = openai_client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        max_tokens=200
    )
    return response.choices[0].message.content.strip()
# Example Usage
if __name__ == "__main__":
    index_name = "human_rights"

    # Sample PDF file paths (change to actual paths)
    pdf_files = ["/home/kiwitech/Downloads/human_rights.pdf"]

    # Index PDF documents with vector embeddings
    for idx, pdf_path in enumerate(pdf_files):
        if os.path.exists(pdf_path):
            index_pdf(index_name, str(idx + 1), pdf_path)
        else:
            print(f"File not found: {pdf_path}")

    # Query for retrieval
    query_text = "What is the role of the United Nations in human rights?"
    retrieved_docs = retrieve_documents(index_name, query_text)

    # Generate AI response
    answer = generate_answer(query_text, retrieved_docs)
    
    print("\n--- Retrieved Documents ---")
    print(json.dumps(retrieved_docs, indent=2),)

    print("\n--- AI-Generated Answer ---")
    print(f"Answer: {answer}\nScore: {score}\n")

Indexed PDF: /home/kiwitech/Downloads/human_rights.pdf (ID: 1)

--- Retrieved Documents ---
[
  {
    "content": "Universal Declaration of Human Rights \nPreamble \nWhereas recognition of the inherent dignity and of the equal and inalienable \nrights of all members of the human family is the foundation of freedom, justice \nand peace in the world,  \nWhereas disregard and contempt for human rights have resulted in barbarous \nacts which have outraged the conscience of mankind, and the advent of a w\norld \nin which human beings shall enjoy freedom of speech and belief and freedom \nfrom fear and want has been proclaimed as the highest aspiration of the common \npeople,  \nWhereas it is essential, if man is not to be compelled to have recourse, as a last \nresort, to rebellion against tyranny and oppression, that human \nrights should be \nprotected by the rule of law,  \nWhereas it is essential to promote the development of fri endly relations between \nnations,  \nWhereas the peoples 